In [45]:
import pandas as pd
import csv
from webcrawl import WebCrawl

In [2]:
# Read movies without Box office
movie = pd.read_csv('movie_without_boxoffice.csv')

In [3]:
len(movie[(movie.YEAR >= 1990) & (movie.YEAR <= 2015) & (movie.COUNTRY.str.contains('USA') \
                                                       & (movie.LANGUAGE.str.contains('English')))])

10151

In [4]:
# filering CSV
filteringCSV = movie[(movie.YEAR >= 1990) & (movie.YEAR <= 2015) & (movie.COUNTRY.str.contains('USA') \
                                                       & (movie.LANGUAGE.str.contains('English')))]

In [5]:
requiredDetails = pd.DataFrame(filteringCSV)

In [9]:
# To check for mutiple IMDB ID's
# requiredDetails['IMDB ID'].value_counts()

In [16]:
# len(requiredDetails)

In [10]:
# One IMDB ID multiple entry found 'tt2279864'
requiredDetails[requiredDetails['IMDB ID'] == 'tt2279864']

,TMDB ID,IMDB ID,TITLE,YEAR,GENRE,RATING,RELEASED,ACTORS,AWARDS,COUNTRY,LANGUAGE
21296,0,tt2279864,Clear History,2013,Comedy,6.5,10 Aug 2013,"Larry David, Bill Hader, Philip Baker Hall, Jo...",NaN,USA,English
25630,133790,tt2279864,Clear History,2013,Comedy,6.5,10 Aug 2013,"Larry David, Bill Hader, Philip Baker Hall, Jo...",NaN,USA,English


In [12]:
# Getting the correct row of IMDB ID 'tt2279864'
idtt2279864 = requiredDetails.loc[25630]

In [13]:
# Dataframe without IMDB ID 'tt2279864'
without_tt2279864 = requiredDetails[requiredDetails['IMDB ID'] != 'tt2279864']

In [25]:
len(without_tt2279864)

10149

In [35]:
# Appending the single entry of IMDB ID 'tt2279864' to Dataframe without IMDB ID 'tt2279864'
finalWithoutBoxoffice = without_tt2279864.append(idtt2279864, ignore_index=True)

In [37]:
# len(finalWithoutBoxoffice)

10150

In [39]:
# No multiple entries of IMDB ID found
# finalWithoutBoxoffice['IMDB ID'].value_counts()

In [40]:
# Converting Dataframe to csv
finalWithoutBoxoffice.to_csv('finalWithoutBoxoffice.csv',index=False)


In [41]:
# Reading correct csv without multilple entrie of IDMDB ID
movie = pd.read_csv('finalWithoutBoxoffice.csv') 

In [43]:
# len(movie)

In [72]:
# List with index 0 contains IMDB ID and index 1 contains TMDB ID
listWithBoxOffice = [[],[]]

In [73]:
# function to extract Box Office by Web Crawling
def extract_boxOffice(tmdbid,imdbid):
    # Extract Box office by crawling IMDB page using IMDB ID
    boxOffice = WebCrawl().extractBoxOfficeByIMDB(imdbid)
    
    if boxOffice == 'N/A':
        # If 'N/A in box office, crawl TMDB web page by using TMDB ID
        boxOffice = WebCrawl().extractBoxOfficeByTMDB(str(tmdbid))
    
    if boxOffice != 'N/A':    
        listWithBoxOffice[0].append(imdbid)
        listWithBoxOffice[1].append(boxOffice)
    else:
        listWithBoxOffice[0].append(imdbid)
        # Appending NaN to Box office so the column will have uniform data type (float) 
        listWithBoxOffice[1].append(float('nan'))

In [74]:
# Shortcut: Duplicate this file and then crawl in each file 1000 entries and then merge the values
for movieID in movie.values[0:1000]:
    # parameters TMDB ID, IMDB ID
    extract_boxOffice(movieID[0], movieID[1])

WARNING......
WRONG ID: 0
WARNING......
WRONG ID: 0
WARNING......
WRONG ID: 0
WARNING......
WRONG ID: 0
WARNING......
WRONG ID: 0
WARNING......
WRONG ID: 0
WARNING......
WRONG ID: 0
WARNING......
WRONG ID: 0
WARNING......
WRONG ID: 0
WARNING......
WRONG ID: 0


In [75]:
len(listWithBoxOffice[0])

1000

In [76]:
# listWithBoxOffice

In [77]:
# Creating a csv file with 1000 entries
with open('movie_with_boxoffice_1000.csv', 'w') as csvfile:
    fieldnames = ['IMDB ID','BOX OFFICE']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for i in range(len(listWithBoxOffice[0])):
        writer.writerow({'IMDB ID': listWithBoxOffice[0][i], 'BOX OFFICE': listWithBoxOffice[1][i]})